In [1]:
# Fitted Q iteration (Expected SARSA)

In [2]:
# policy_train_path = '../continuous/dqn_normal/dqn_normal_actions_train.p'
# policy_val_path = '../continuous/dqn_normal/dqn_normal_actions_val.p'
# policy_test_path = '../continuous/dqn_normal/dqn_normal_actions_test.p'

In [3]:
import numpy as np
import math
import os
import random
import numpy as np
import pandas as pd
from pandas import DataFrame
import cPickle as pickle
import copy

In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [5]:
with open('../data/state_features.txt') as f:
    state_features = f.read().split()
print (state_features)
print len(state_features)

['Albumin', 'Arterial_BE', 'Arterial_lactate', 'Arterial_pH', 'BUN', 'CO2_mEqL', 'Calcium', 'Chloride', 'Creatinine', 'DiaBP', 'FiO2_1', 'GCS', 'Glucose', 'HCO3', 'HR', 'Hb', 'INR', 'Ionised_Ca', 'Magnesium', 'MeanBP', 'PT', 'PTT', 'PaO2_FiO2', 'Platelets_count', 'Potassium', 'RR', 'SGOT', 'SGPT', 'SIRS', 'SOFA', 'Shock_Index', 'Sodium', 'SpO2', 'SysBP', 'Temp_C', 'Total_bili', 'WBC_count', 'Weight_kg', 'age', 'elixhauser', 'gender', 'mechvent', 'output_4hourly', 'output_total', 'paCO2', 'paO2', 're_admission', 'bloc']
48


In [6]:
df = pd.read_csv('../data/rl_train_data_final_cont.csv')

In [7]:
df.head()

,bloc,icustayid,charttime,gender,age,elixhauser,re_admission,died_in_hosp,mortality_90d,Weight_kg,...,median_dose_vaso,max_dose_vaso,input_total_tev,input_4hourly_tev,output_total,output_4hourly,cumulated_balance_tev,vaso_input,iv_input,reward
0,0.000000,3,7245052800,0.0,0.412568,0.0,0.0,0,1,0.262712,...,0.0,0.0,0.797351,0.939195,0.589916,0.750908,0.554500,0.0,4.0,0.125000
1,0.222560,3,7245067200,0.0,0.412568,0.0,0.0,0,1,0.262712,...,0.0,0.0,0.831780,0.934543,0.674384,0.819589,0.580033,0.0,4.0,0.657321
2,0.356608,3,7245081600,0.0,0.412568,0.0,0.0,0,1,0.262712,...,0.0,0.0,0.833222,0.656575,0.765423,0.939329,0.555033,0.0,2.0,1.367788
3,0.452837,3,7245096000,0.0,0.412568,0.0,0.0,0,1,0.262712,...,0.0,0.0,0.834033,0.603831,0.783597,0.847073,0.545700,0.0,2.0,1.199099
4,0.527957,3,7245110400,0.0,0.412568,0.0,0.0,0,1,0.262712,...,0.0,0.0,0.834836,0.603831,0.794059,0.811583,0.539533,0.0,2.0,1.057596


In [8]:
val_df = pd.read_csv('../data/rl_val_data_final_cont.csv')

In [9]:
test_df = pd.read_csv('../data/rl_test_data_final_cont.csv')

In [10]:
# # Here we load the actions for the policy we want to evaluate into the relevant dataframes
# policy_train = pickle.load(open(policy_train_path, "rb" ))
# policy_test = pickle.load(open(policy_test_path, "rb" ))
# policy_val = pickle.load(open(policy_val_path, "rb" ))

# df['policy'] = np.array(policy_train)
# test_df['policy'] = np.array(policy_test)
# val_df['policy'] = np.array(policy_val)


save_dir = './MixNN_EM/'
train_actions = np.load(save_dir + "cond_lik_train_dr.npy")
val_actions = np.load(save_dir + "cond_lik_val_dr.npy")
test_actions = np.load(save_dir + "cond_lik_test_dr.npy")

mix_comp_eval = 3 # SET THIS ACCORDINGLY
train_actions = [np.array(list(i)) for i in train_actions[:,:,mix_comp_eval]]
val_actions = [np.array(list(i)) for i in val_actions[:,:,mix_comp_eval]]
test_actions = [np.array(list(i)) for i in test_actions[:,:,mix_comp_eval]]

df['policy'] = train_actions
val_df['policy'] = val_actions
test_df['policy'] = test_actions

In [11]:
fq_state_terms = copy.deepcopy(state_features)
fq_state_terms.append('iv_input')
fq_state_terms.append('vaso_input')

In [12]:
# define an action mapping - how to get an id representing the action from the (iv,vaso) tuple
action_map = {}
count = 0
for iv in range(5):
    for vaso in range(5):
        action_map[(iv,vaso)] = count
        count += 1

In [13]:
def get_exp_next_q(state, prob_actions, reg):
    # state excludes the actions
    target = 0.0
    state = list(state)
    state.append(0)
    state.append(0)
    state = np.array(state)
    for iv in range(5):
        for vaso in range(5):
            index = action_map[(iv,vaso)]
            # compute p(a')Q(s'a'))
            state[-2] = iv
            state[-1] = vaso
            target += reg.predict(state.reshape(1,-1)) * prob_actions[index]
    return target

In [14]:
gamma = 0.99
def make_inp_op(df_in, reg):
    # for each (s,a) in train set, compute target Q value for eval policy
    X  = []
    y = []
    
    df_local = df_in.copy()
    
    df_local = df_local.reset_index()
    
    for count, i in enumerate(df_local.index):
        if count % 10000 == 0 and count > 0:
            print "make_inp_op, step %d" % count
        if i != df_local.index[-1]:
            cur_x = df_local.loc[i, fq_state_terms]
            if df_local.loc[i, 'icustayid']  == df_local.loc[i+1, 'icustayid']:
                if reg is None:
                    cur_y = df_local.loc[i, 'reward']
                else:
                    # compute r + gamma* expectation(Q(s',a'))
                    next_state = df_in.loc[i+1, state_features]
                    action_prob = df_in.loc[i+1, 'policy']
                    exp_q = gamma * get_exp_next_q(next_state, action_prob,reg)
                    cur_y = df_local.loc[i, 'reward'] + gamma * exp_q
            else:
                cur_y = df_local.loc[i, 'reward'] 
            X.append(cur_x)
            y.append(cur_y)
    return np.array(X), np.array(y)
    
    

In [92]:
# fitted q iteration
# tmp_df = df.iloc[:100]
from sklearn.linear_model import SGDRegressor

# make the initial training inp and op
# reg = None

num_iters = 10

for i in range(num_iters):
    if reg is None:
        train_X, train_y = make_inp_op(df, None)
    else:
        train_X, train_y = make_inp_op(df, reg)
    # create new reg object, and then call fit on train_X and train_y
    reg = SGDRegressor(n_iter=100)
    reg.fit(train_X, train_y)

    val_X, val_y = make_inp_op(val_df, reg)
    pred = reg.predict(val_X)
    mean_error = np.mean((pred - val_y)**2)
    print "Mean on val set is %f, mse is %f" % (np.mean(pred),mean_error)
    


make_inp_op, step 10000
make_inp_op, step 20000
make_inp_op, step 30000
make_inp_op, step 40000
make_inp_op, step 50000
make_inp_op, step 60000
make_inp_op, step 70000
make_inp_op, step 80000
make_inp_op, step 90000
make_inp_op, step 100000
make_inp_op, step 110000
make_inp_op, step 120000
make_inp_op, step 130000
make_inp_op, step 140000
make_inp_op, step 150000
make_inp_op, step 160000
make_inp_op, step 10000
make_inp_op, step 20000
Mean on val set is 7.479799, mse is 6.726965
make_inp_op, step 10000
make_inp_op, step 20000
make_inp_op, step 30000
make_inp_op, step 40000
make_inp_op, step 50000
make_inp_op, step 60000
make_inp_op, step 70000
make_inp_op, step 80000
make_inp_op, step 90000
make_inp_op, step 100000
make_inp_op, step 110000
make_inp_op, step 120000
make_inp_op, step 130000
make_inp_op, step 140000
make_inp_op, step 150000
make_inp_op, step 160000
make_inp_op, step 10000
make_inp_op, step 20000
Mean on val set is 7.911850, mse is 6.632021
make_inp_op, step 10000
make_inp

In [ ]:
%matplotlib inline
np.mean(reg.predict(train_X))

In [87]:
val_X, val_y = make_inp_op(val_df, None)

make_inp_op, step 10000
make_inp_op, step 20000


In [84]:
np.mean(reg.predict(val_X))

7.1539674240623903

In [ ]:
from sklearn.externals import joblib
joblib.dump(reg, 'lr_model.pkl')

In [ ]:
# LOAD STUFF HERE SEPARATELY

In [22]:
def final_pred(df_in, reg):
    # for each (s,a) in set, compute Q value under target policy
    Q = []
    
    df_local = df_in.copy()
    
    df_local = df_local.reset_index()
    
    for count, i in enumerate(df_local.index):
        if count % 10000 == 0 and count > 0:
            print "final_pred, step %d" % count
        if i != df_local.index[-1]:
            cur_x = df_local.loc[i, fq_state_terms]
            if df_local.loc[i, 'icustayid']  == df_local.loc[i+1, 'icustayid']:
                cur_q = reg.predict(cur_x.reshape(1,-1))
            else:
                cur_q = df_local.loc[i, 'reward'] 
            Q.append(cur_q)
        else:
            Q.append(df_local.loc[i, 'reward'] )
    return np.array(Q)
    
        

In [23]:
reg = joblib.load('lr_model.pkl')

In [24]:
# predict Q on all the train, test, val sets
val_q = final_pred(val_df, reg)
test_q = final_pred(test_df, reg)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  from ipykernel import kernelapp as app


final_pred, step 10000
final_pred, step 20000
final_pred, step 10000
final_pred, step 20000
final_pred, step 30000
final_pred, step 40000


In [26]:
np.save('val_q_learned_policy.npy', val_q)
np.save('test_q_learned_policy.npy', test_q)